In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_squared_error
import cv2

In [2]:
results_dir = "../results"
dataset_dir = "../datasets/lacbed"

In [4]:
space_groups = pd.read_csv("space_groups.csv")

data = {"code": [], "group": [], "seed": [], "phase": [], "epoch": [], "MSE": []}

for seed in os.listdir(results_dir):
    for output in os.listdir(os.path.join(results_dir, seed)):
        phase, epoch = output.split("_")
        rms = 0
        for image in os.listdir(os.path.join(results_dir, seed, output, "images")):
            code = image.split("_")[0]
            group = space_groups.loc[space_groups['ICSD_code'] == int(code), 'space_group'].iloc[0]
            
            fake = cv2.cvtColor(cv2.imread(
                os.path.join(results_dir, seed, output, "images", image)
            ), cv2.COLOR_BGR2GRAY) / 255
            real = cv2.cvtColor(cv2.imread(
                os.path.join(dataset_dir, code, code + "_+0+0+0.png")
            ), cv2.COLOR_BGR2GRAY) / 255
            
            MSE = mean_squared_error(fake, real)
            
            data["code"].append(code)
            data["group"].append(group)
            data["seed"].append(seed)
            data["phase"].append(phase)
            data["epoch"].append(epoch)
            data["MSE"].append(MSE)

        df_cache = pd.DataFrame(data)
        df_cache.to_csv("data_cache.csv")

df = pd.DataFrame(data)
df.to_csv("data.csv")

In [38]:
sg = pd.read_csv("space_groups.csv")
sg

,ICSD_code,space_group
0,1000,F m -3 m
1,100018,P -4 3 n
2,10009,F -4 3 m
3,100098,I 2 3
4,100099,I 2 3
...,...,...
12449,9988,F 41 3 2
12450,99963,P m -3 m
12451,99964,P m -3 m
12452,99965,P m -3 m


In [59]:
groups = set(sg["space_group"].to_list())
groups

{'F -4 3 c',
 'F -4 3 m',
 'F 2 3',
 'F 4 3 2',
 'F 41 3 2',
 'F d -3 S',
 'F d -3 Z',
 'F d -3 c Z',
 'F d -3 m S',
 'F d -3 m Z',
 'F m -3',
 'F m -3 c',
 'F m -3 m',
 'I -4 3 d',
 'I -4 3 m',
 'I 2 3',
 'I 21 3',
 'I 4 3 2',
 'I 41 3 2',
 'I a -3',
 'I a -3 d',
 'I m -3',
 'I m -3 m',
 'P -4 3 m',
 'P -4 3 n',
 'P 2 3',
 'P 21 3',
 'P 4 3 2',
 'P 41 3 2',
 'P 42 3 2',
 'P 43 3 2',
 'P a -3',
 'P m -3',
 'P m -3 m',
 'P m -3 n',
 'P n -3 S',
 'P n -3 Z',
 'P n -3 m S',
 'P n -3 m Z',
 'P n -3 n Z'}

In [62]:
sg = pd.read_csv("space_groups.csv")

groups = set(sg["space_group"].to_list())

differences = {group: np.zeros((128, 128)) for group in groups}
differences_counts = {group: 0 for group in groups}
count = 0

for seed in os.listdir(results_dir):
    for output in os.listdir(os.path.join(results_dir, seed)):
        phase, epoch = output.split("_")
        rms = 0
        for image in os.listdir(os.path.join(results_dir, seed, output, "images")):
            
            if count > 10:
                break
            
            code = image.split("_")[0]
            
            fake = cv2.cvtColor(cv2.imread(
                os.path.join(results_dir, seed, output, "images", image)
            ), cv2.COLOR_BGR2GRAY) / 255
            real = cv2.cvtColor(cv2.imread(
                os.path.join(dataset_dir, code, code + "_+0+0+0.png")
            ), cv2.COLOR_BGR2GRAY) / 255
            
            group = sg.loc[sg['ICSD_code'] == int(code), 'space_group'].iloc[0]
            
            differences[group] += (real - fake)
            differences_counts[group] += 1
            # print(differences[sg.loc[sg['ICSD_code'] == int(code), 'space_group'].iloc[0]])
            
            count += 1

            
            # MSE = mean_squared_error(fake, real)
            
            # data["code"].append(code)
            # data["seed"].append(seed)
            # data["phase"].append(phase)
            # data["epoch"].append(epoch)
            # data["MSE"].append(MSE)

for group in groups:
    differences[group] /= differences_counts[group]

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[-0.05882353 -0.03921569 -0.04705882 ... -0.04705882 -0.05490196
  -0.06666667]
 [-0.04705882 -0.05490196 -0.03921569 ... -0.03921569 -0.04705882
  -0.0627451 ]
 [-0.04705882 -0.03137255 -0.04705882 ... -0.03921569 -0.03921569
  -0.05490196]
 ...
 [-0.04705882 -0.03137255 -0.03921569 ... -0.03921569 -0.03137255
  -0.03921569]
 [-0.05490196 -0.05490196 -0.03137255 ... -0.03921569 -0.03137255
  -0.05490196]
 [-0.05882353 -0.0627451  -0.05098039 ... -0.04313725 -0.05490196
  -0.06666667]]
[[0.18039216 0.18039216 0.18823529 ... 0